In [39]:
import numpy as np
import matplotlib.pyplot as plt
from bokeh.plotting import figure
from bokeh.io import output_file, show
from bokeh.embed import components
from bokeh.io import curdoc
from bokeh.layouts import widgetbox
from bokeh.models import ColumnDataSource, Slider
from bokeh.layouts import column, row
import scipy as sp
import scipy.special
import math
from ipywidgets import interact
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
output_notebook()

def get_order(a=1, lambDa=1):
    """
    Calculate the order of the integration based on radius of the sphere
    and the wavelength of the incident field

    Parameters
    ----------
        a: radius of the sphere, float
        lambDa: wavelength of the incident field, float

    Returns
    -------
        l: order of the field, 1-D array int
    """

    # calculate the maximal order based on the Bessel function decaying
    l_max = math.ceil(2*np.pi*a/lambDa + 4*(2*np.pi*a/lambDa)**(1/3) + 2)

    # create a order vector from the maxiaml order
    l = np.arange(0, l_max+1, 1)

    return l


def coeff_b(l, k, n=1, a=1):

    """
    Calculate the B vector with respect to the sphere properties

    Note that B vector is independent to scatter matrix and only
    relys on n and a

    Parameters
    ----------
        l: order vector of the simulation, 1-D array, int
        k: wavenumber of the incident field, float
        n: refractive index (and attenuation coeff.) of the sphere, complex
        a: radius of the sphere, float

    Returns
    -------
        B: B coefficient vector, 1-D array, complex
    """

    # calculate everything related to spherical Bessel function of the 1st kind
    jka = sp.special.spherical_jn(l, k * a)
    jka_p = sp.special.spherical_jn(l, k * a, derivative=True)
    jkna = sp.special.spherical_jn(l, k * n * a)
    jkna_p = sp.special.spherical_jn(l, k * n * a, derivative=True)

    # calculate everything related to spherical Bessel funtion of the 2nd kind
    yka = sp.special.spherical_yn(l, k * a)
    yka_p = sp.special.spherical_yn(l, k * a, derivative=True)

    # calculate spherical Hankel function of the 1st kind and its derivative
    hka = jka + yka * 1j
    hka_p = jka_p + yka_p * 1j

    # calculate different terms of B
    bi = jka * jkna_p * n
    ci = jkna * jka_p
    di = jkna * hka_p
    ei = hka * jkna_p * n

    # calculate B
    B = (bi - ci) / (di - ei)

    return B

def update(a, n):
    l = get_order(10, lambDa)
    r.data_source.data['x'] = l
    r.data_source.data['y'] = np.real(coeff_b(l, k, n, a))
    
    push_notebook()

a = 10
lambDa = 1
n = 1.5
k = 2*np.pi / lambDa

l = get_order(a, lambDa)

B = np.real(coeff_b(l, k, n, a))

x = l
y = B
p = figure(x_axis_label = 'order',
           y_axis_label = 'a.u.')
r = p.line(x, y)

show(p, notebook_handle=True)
interact(update, a=(1,10), n=(1, 1.5, 0.1))

Loading BokehJS ...

interactive(children=(IntSlider(value=5, description='a', max=10, min=1), FloatSlider(value=1.2, description='…

<function __main__.update(a, n)>